# Main analysis of data
## This notebook is intended to be a structured results collection from tools developed locally
### more infoo



---


# Statistics
---
### Envelope enhancer


In [11]:
# main loop
import pathlib
import os

# iterate over file objects
main_dir = pathlib.Path("enhanced_fft/envelope_enhancer")
for root, _, files in os.walk(main_dir):
    for file_name in files:
        file = pathlib.Path(root, file_name)
        print(file)



enhanced_fft/envelope_enhancer/record_18/ac_1/z/part_0.csv
enhanced_fft/envelope_enhancer/record_18/ac_1/y/part_0.csv
enhanced_fft/envelope_enhancer/record_18/ac_1/y/part_1.csv
enhanced_fft/envelope_enhancer/record_18/ac_1/x/part_0.csv
enhanced_fft/envelope_enhancer/record_18/ac_1/x/part_1.csv
enhanced_fft/envelope_enhancer/record_18/ac_2/z/part_0.csv
enhanced_fft/envelope_enhancer/record_18/ac_2/z/part_1.csv
enhanced_fft/envelope_enhancer/record_18/ac_2/y/part_0.csv
enhanced_fft/envelope_enhancer/record_18/ac_2/y/part_2.csv
enhanced_fft/envelope_enhancer/record_18/ac_2/y/part_1.csv
enhanced_fft/envelope_enhancer/record_18/ac_2/y/part_3.csv
enhanced_fft/envelope_enhancer/record_18/ac_2/x/part_0.csv
enhanced_fft/envelope_enhancer/record_18/ac_2/x/part_1.csv
enhanced_fft/envelope_enhancer/record_12/ac_1/z/part_0.csv
enhanced_fft/envelope_enhancer/record_12/ac_1/z/part_2.csv
enhanced_fft/envelope_enhancer/record_12/ac_1/z/part_1.csv
enhanced_fft/envelope_enhancer/record_12/ac_1/z/part_3.c